<a href="https://colab.research.google.com/github/xmks-colab/SmartCutter/blob/main/SmartStitchConsole_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


#@title SmartStitch { display-mode: "form" }
import ipywidgets as widgets
from IPython.display import display
import subprocess
import os
from google.colab import files
import shutil
import sys

def run_command(cmd, silent=True):
    """Executa comando silenciosamente"""
    try:
        if silent:
            result = subprocess.run(cmd, shell=True, capture_output=True, text=True, check=True)
        else:
            result = subprocess.run(cmd, shell=True, check=True)
        return True, result
    except subprocess.CalledProcessError as e:
        return False, e

def install_smartstitch():
    print("Preparando...")

    # Verifica se o diretório já existe
    if os.path.exists('SmartStitch'):
        print("SmartStitch já está instalado. Ignorando...")
        os.chdir('SmartStitch')
        print("Finalizado.")
        return

    # Clone do repositório
    success, _ = run_command("git clone https://github.com/MechTechnology/SmartStitch.git")
    if not success:
        print("Erro ao clonar o repositório.")
        return

    # Mudança para o diretório
    os.chdir('SmartStitch')

    # Instalação das dependências
    packages = ["numpy", "pillow", "natsort", "psd-tools"]
    for package in packages:
        success, _ = run_command(f"pip install {package}")
        if not success:
            print(f"Erro ao instalar {package}.")
            return

    print("Finalizado.")

# Executar instalação
install_smartstitch()

# --- Criar pasta input automaticamente ---
def create_input_folder():
    input_path = "/content/SmartStitch/input"
    os.makedirs(input_path, exist_ok=True)
    return input_path

# --- pasta imutavel ---
input_folder = "input"

split_height = widgets.IntText(
    value=5000,
    description="Split H:",
    layout=widgets.Layout(width="40%")
)

output_format = widgets.Dropdown(
    options=[".png", ".jpg", ".webp", ".bmp", ".psd", ".tiff", ".tga"],
    value=".png",
    description="Formato:"
)

custom_width = widgets.IntText(
    value=0,
    description="Largura:",
    layout=widgets.Layout(width="30%")
)

detection_type = widgets.Dropdown(
    options=["pixel", "none"],
    value="pixel",
    description="Detecção:"
)

sensitivity = widgets.IntSlider(
    value=90, min=0, max=100, step=1,
    description="Sensib:"
)

quality = widgets.IntSlider(
    value=100, min=1, max=100, step=1,
    description="Qualidade:"
)

ignorable_pixels = widgets.IntText(
    value=5,
    description="Ign Pixels:"
)

scan_line = widgets.IntSlider(
    value=5, min=1, max=100, step=1,
    description="Scan Line:"
)

# --- Checkbox para zip e download ---
zip_download = widgets.Checkbox(
    value=True,
    description="Zipar e Baixar",
    indent=False
)

# --- 2. Saída e Função de Upload ---
output = widgets.Output()

def upload_images(b):
    output.clear_output()
    with output:
        input_path = '/content/SmartStitch/input'
        print(f"📁 Usando pasta input: {input_path}")

        print("📤 Selecione as imagens para upload...")

        original_cwd = os.getcwd()
        try:
            os.chdir(input_path)
            uploaded = files.upload()
        finally:
            os.chdir(original_cwd)

        if uploaded:
            print(f"📥 {len(uploaded)} arquivo(s) salvo(s) diretamente na pasta input...")
            for filename in uploaded.keys():
                dst_path = f"{input_path}/{filename}"
                print(f"✅ {filename} → {dst_path}")

            print(f"🎉 Upload concluído! {len(uploaded)} imagem(s) salvas em {input_path}")
        else:
            print("❌ Nenhum arquivo foi selecionado")

def run_smartstitch(b):
    output.clear_output()

    # Garantir que a pasta existe antes de executar
    input_path = create_input_folder()

    with output:
        # Informação sobre imagens (opcional, apenas para informar o usuário)
        if os.path.exists(input_path):
            images = [f for f in os.listdir(input_path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.webp', '.tiff', '.tga', '.psd'))]
            if images:
                print(f"📸 Encontradas {len(images)} imagem(s): {', '.join(images[:3])}{'...' if len(images) > 3 else ''}")

    cmd = [
        "python", "SmartStitchConsole.py",
        "-i", input_folder,
        "-sh", str(split_height.value),
        "-t", output_format.value
    ]

    if custom_width.value > 0:
        cmd += ["-cw", str(custom_width.value)]
    if detection_type.value:
        cmd += ["-dt", detection_type.value]
    if sensitivity.value:
        cmd += ["-s", str(sensitivity.value)]
    if quality.value:
        cmd += ["-lq", str(quality.value)]
    if ignorable_pixels.value:
        cmd += ["-ip", str(ignorable_pixels.value)]
    if scan_line.value:
        cmd += ["-sl", str(scan_line.value)]

    with output:
        print(f"\n🚀 Executando: {' '.join(cmd)}")
        print("-" * 50)
        process = subprocess.run(cmd, text=True, capture_output=True)

        if process.stdout.strip():
            print("📋 Output:")
            print(process.stdout)

        if process.stderr.strip():
            print("⚠️ Avisos/Erros:")
            print(process.stderr)

        print("-" * 50)
        print("✅ Processamento concluído!")

        # Zip e download se checkbox ativado
        if zip_download.value:
            output_folder = "/content/SmartStitch/input [stitched]"
            if os.path.exists(output_folder):
                zip_path = "/content/SmartStitch/stitched_results"
                shutil.make_archive(zip_path, 'zip', output_folder)
                zip_file = f"{zip_path}.zip"
                print(f"📦 Zip criado: {zip_file}")
                print("⬇️ Iniciando download...")
                files.download(zip_file)
            else:
                print(f"❌ Pasta de saída não encontrada: {output_folder}")

def clear_input_folder(b):
    output.clear_output()
    input_path = '/content/SmartStitch/input'
    with output:
        if os.path.exists(input_path):
            for filename in os.listdir(input_path):
                file_path = os.path.join(input_path, filename)
                try:
                    if os.path.isfile(file_path) or os.path.islink(file_path):
                        os.unlink(file_path)
                    elif os.path.isdir(file_path):
                        shutil.rmtree(file_path)
                except Exception as e:
                    print(f"❌ Erro ao limpar {file_path}: {e}")
            print("✅ Pasta de entrada limpa!")
        else:
            print("❌ Pasta de entrada não existe!")

def clear_output_folder(b):
    output.clear_output()
    output_folder = '/content/SmartStitch/input [stitched]'
    with output:
        if os.path.exists(output_folder):
            shutil.rmtree(output_folder)
            print("✅ Pasta de saída limpa!")
        else:
            print("❌ Pasta de saída não existe!")

# --- 3. Botões ---
upload_button = widgets.Button(
    description="📤 Upload",
    button_style="info",
    tooltip="Enviar imagens para a pasta input"
)
upload_button.on_click(upload_images)

run_button = widgets.Button(
    description="▶️ Executar",
    button_style="success",
    tooltip="Executar SmartStitch com as configurações atuais"
)
run_button.on_click(run_smartstitch)

clear_input_button = widgets.Button(
    description="Lpar Entrada",
    button_style="danger",
    tooltip="Limpar a pasta de entrada"
)
clear_input_button.on_click(clear_input_folder)

clear_output_button = widgets.Button(
    description="Lpar Saida",
    button_style="danger",
    tooltip="Limpar a pasta de saida"
)
clear_output_button.on_click(clear_output_folder)

# --- 4. Montar Layout ---
ui = widgets.VBox([
    widgets.HTML("<h3>🧵 SmartStitch Console GUI</h3>"),
    widgets.HTML("<h4>⚙️ Configurações</h4>"),
    widgets.HBox([split_height, custom_width]),
    widgets.HBox([output_format, detection_type]),

    widgets.HTML("<h4>🎚️ Parâmetros Avançados</h4>"),
    sensitivity,
    quality,
    ignorable_pixels,
    scan_line,

    widgets.HTML("<h4>🎬 Execução</h4>"),
    widgets.HBox([upload_button, run_button, clear_input_button, clear_output_button]),
    zip_download,

    widgets.HTML("<h4>📊 Output</h4>"),
    output
])

# Criar a pasta input automaticamente ao inicializar
print("🔧 Inicializando SmartStitch GUI...")
create_input_folder()
#print("✅ Pasta input criada em /content/SmartStitch/input")

display(ui)

In [ ]:

#@title Gerencidor de arquivos { display-mode: "form" }
import os
import shutil
import ipywidgets as widgets
from IPython.display import display, clear_output, Javascript
import threading

class FileManager:
    def __init__(self, start_path='/content'):
        print("Menu popup com: Apagar, Mover, Copiar, Renomear, Copiar caminho")
        self.current_path = start_path
        self.output = widgets.Output()
        self.path_label = widgets.Label()
        self.status_label = widgets.Label()
        self.progress_label = widgets.Label()
        self.item_to_move = None
        self.item_to_copy = None
        self.multiselect_mode = False
        self.selected_items = set()
        self.item_buttons = {}  # Cache dos botões para atualização rápida
        self.refresh()

    def list_directory(self, path):
        try:
            return sorted(os.listdir(path))
        except Exception as e:
            return [f"Error: {e}"]

    def change_directory(self, folder):
        if not self.multiselect_mode:
            new_path = os.path.join(self.current_path, folder)
            if os.path.isdir(new_path):
                self.current_path = new_path
                self.refresh()

    def go_up(self):
        if not self.multiselect_mode:
            parent = os.path.dirname(self.current_path)
            if os.path.exists(parent):
                self.current_path = parent
                self.refresh()

    def toggle_multiselect(self):
        self.multiselect_mode = not self.multiselect_mode
        if not self.multiselect_mode:
            self.selected_items.clear()
        self.set_status(f"{'✅ Multiseleção ativada' if self.multiselect_mode else '❌ Multiseleção desativada'}")
        self.refresh()

    def select_all_items(self):
        items = [item for item in self.list_directory(self.current_path) if not item.startswith("Error:")]
        self.selected_items.update(items)
        self.set_status(f"✅ {len(items)} itens selecionados!")
        self._update_path_label()  # Atualiza apenas o label do path
        # Atualiza visualmente todos os botões
        for item in items:
            if item in self.item_buttons:
                self._update_item_button(item, True)

    def toggle_item_selection(self, item):
        """Método otimizado que não re-renderiza toda a interface"""
        if item in self.selected_items:
            self.selected_items.remove(item)
            is_selected = False
        else:
            self.selected_items.add(item)
            is_selected = True

        # Atualiza apenas o botão específico
        self._update_item_button(item, is_selected)
        # Atualiza apenas o contador no path
        self._update_path_label()

    def _update_item_button(self, item, is_selected):
        """Atualiza apenas o botão específico do item"""
        if item in self.item_buttons:
            full_path = os.path.join(self.current_path, item)
            is_dir = os.path.isdir(full_path)
            new_label = f"{'☑️' if is_selected else '☐'} {'📁' if is_dir else '📄'} {item}"

            button = self.item_buttons[item]
            button.description = new_label
            button.button_style = 'info' if is_selected else ''

    def _update_path_label(self):
        """Atualiza apenas o label do caminho com contador de seleção"""
        path_text = f"📁 {self.current_path}"
        if self.multiselect_mode:
            path_text += f" | ☑️ Multiseleção ({len(self.selected_items)} selecionados)"
        self.path_label.value = path_text

    def _create_input_dialog(self, title, placeholder, value="", callback=None):
        """Cria dialogo de entrada genérico"""
        name_input = widgets.Text(value=value, placeholder=placeholder, description='Nome:',
                                style={'description_width': 'initial'})
        confirm_btn = widgets.Button(description='✅ Confirmar', button_style='success')
        cancel_btn = widgets.Button(description='❌ Cancelar')

        def on_confirm(btn):
            if callback:
                callback(name_input.value.strip())

        confirm_btn.on_click(on_confirm)
        cancel_btn.on_click(lambda btn: self.refresh())

        with self.output:
            clear_output()
            self.path_label.value = f"📁 {self.current_path}"
            display(widgets.VBox([self.path_label, self.status_label, self.progress_label, widgets.HTML(f"<h4>{title}</h4>"),
                                name_input, widgets.HBox([confirm_btn, cancel_btn])]))

    def create_folder(self):
        def do_create(folder_name):
            if folder_name:
                try:
                    os.makedirs(os.path.join(self.current_path, folder_name), exist_ok=False)
                    self.set_status(f"✅ Pasta '{folder_name}' criada com sucesso!")
                except FileExistsError:
                    self.set_status(f"❌ Pasta '{folder_name}' já existe!")
                except Exception as e:
                    self.set_status(f"❌ Erro ao criar pasta: {e}")
            else:
                self.set_status("❌ Nome da pasta não pode estar vazio!")
            self.refresh()

        self._create_input_dialog("📁 Criar Nova Pasta", "Digite o nome da pasta", callback=do_create)

    def rename_item(self, item):
        def do_rename(new_name):
            if new_name and new_name != item:
                try:
                    os.rename(os.path.join(self.current_path, item),
                             os.path.join(self.current_path, new_name))
                    self.set_status(f"✅ Renomeado de '{item}' para '{new_name}'!")
                except FileExistsError:
                    self.set_status(f"❌ Já existe um item com o nome '{new_name}'!")
                except Exception as e:
                    self.set_status(f"❌ Erro ao renomear: {e}")
            elif not new_name:
                self.set_status("❌ Nome não pode estar vazio!")
            self.refresh()

        self._create_input_dialog(f"✏️ Renomear: {item}", "Digite o novo nome", item, do_rename)

    def _create_menu(self, title, buttons_config):
        """Cria menu genérico com botões configuráveis"""
        buttons = []
        for config in buttons_config:
            btn = widgets.Button(description=config['desc'], button_style=config.get('style', ''),
                               layout=widgets.Layout(width='200px', margin='2px'))
            btn.on_click(config['callback'])
            buttons.append(btn)

        with self.output:
            clear_output()
            self.path_label.value = f"📁 {self.current_path}"
            display(widgets.VBox([self.path_label, self.status_label, self.progress_label, widgets.HTML(f"<h4>{title}</h4>"),
                                widgets.VBox(buttons, layout=widgets.Layout(border='1px solid #ddd',
                                                                         padding='10px', margin='10px'))]))

    def show_item_menu(self, item):
        if self.multiselect_mode:
            self.show_multiselect_menu()
            return

        full_path = os.path.join(self.current_path, item)

        def copy_path(btn):
            display(Javascript(f'navigator.clipboard.writeText("{full_path}")'))
            self.set_status(f"📋 Caminho copiado: {os.path.basename(full_path)}")
            self.refresh()

        buttons = [
            {'desc': '🗑️ Apagar', 'style': 'danger', 'callback': lambda btn: self.delete_item(item)},
            {'desc': '📦 Mover', 'callback': lambda btn: self.mark_for_move(item)},
            {'desc': '📄 Copiar', 'style': 'warning', 'callback': lambda btn: self.mark_for_copy(item)},
            {'desc': '✏️ Renomear', 'style': 'primary', 'callback': lambda btn: self.rename_item(item)},
            {'desc': '📋 Copiar Caminho', 'style': 'info', 'callback': copy_path},
            {'desc': '☑️ Multiseleção', 'style': 'info', 'callback': lambda btn: self.toggle_multiselect()},
            {'desc': '❌ Cancelar', 'callback': lambda btn: self.refresh()}
        ]

        self._create_menu(f"📋 Menu: {item}", buttons)

    def show_multiselect_menu(self):
        if not self.selected_items:
            self.set_status("❌ Nenhum item selecionado!")
            self.refresh()
            return

        count = len(self.selected_items)
        buttons = [
            {'desc': f'🗑️ Apagar ({count})', 'style': 'danger', 'callback': lambda btn: self.delete_multiple_items()},
            {'desc': f'📦 Mover ({count})', 'callback': lambda btn: self.mark_multiple_for_move()},
            {'desc': f'📄 Copiar ({count})', 'style': 'warning', 'callback': lambda btn: self.mark_multiple_for_copy()},
            {'desc': '❌ Cancelar', 'callback': lambda btn: self.refresh()}
        ]

        selected_list = ", ".join(list(self.selected_items)[:3])
        if count > 3:
            selected_list += f" e mais {count - 3} itens"

        # Criar botões com eventos já conectados
        button_widgets = []
        for btn_config in buttons:
            btn = widgets.Button(
                description=btn_config['desc'],
                button_style=btn_config.get('style', ''),
                layout=widgets.Layout(width='250px', margin='2px')
            )
            btn.on_click(btn_config['callback'])
            button_widgets.append(btn)

        with self.output:
            clear_output()
            self.path_label.value = f"📁 {self.current_path}"
            display(widgets.VBox([
                self.path_label, self.status_label, self.progress_label,
                widgets.HTML(f"<h4>📋 Menu Multiseleção</h4><p><strong>Selecionados:</strong> {selected_list}</p>"),
                widgets.VBox(button_widgets, layout=widgets.Layout(border='1px solid #ddd', padding='10px', margin='10px'))
            ]))

    def _confirm_delete(self, items, callback):
        """Cria dialogo de confirmação de exclusão"""
        count = len(items) if isinstance(items, list) else 1
        item_display = items if isinstance(items, str) else f"{count} itens"

        confirm = widgets.Button(description=f'❗ Confirmar apagar', button_style='danger')
        cancel = widgets.Button(description='Cancelar')

        confirm.on_click(lambda btn: callback())
        cancel.on_click(lambda btn: self.refresh())

        with self.output:
            clear_output()
            self.path_label.value = f"📁 {self.current_path}"
            content = [self.path_label, self.status_label, self.progress_label, widgets.HTML(f"<h4>🗑️ Confirmar Exclusão</h4>")]

            if isinstance(items, list):
                content.append(widgets.HTML(f"<p>Tem certeza que deseja apagar <strong>{count} itens</strong>?</p>"))
                content.append(widgets.HTML(f"<p><small>{', '.join(items[:5])}{' ...' if count > 5 else ''}</small></p>"))
            else:
                content.append(widgets.HTML(f"<p>Tem certeza que deseja apagar: <strong>{items}</strong>?</p>"))

            content.append(widgets.HBox([confirm, cancel]))
            display(widgets.VBox(content))

    def delete_item(self, item):
        def do_delete():
            full_path = os.path.join(self.current_path, item)
            try:
                (shutil.rmtree if os.path.isdir(full_path) else os.remove)(full_path)
                self.set_status(f"✅ {item} apagado com sucesso!")
            except Exception as e:
                self.set_status(f"❌ Erro ao apagar {item}: {e}")
            self.refresh()

        self._confirm_delete(item, do_delete)

    def delete_multiple_items(self):
        items_list = list(self.selected_items)

        def do_delete():
            success_count = error_count = 0
            total = len(items_list)

            for i, item in enumerate(items_list, 1):
                self._update_progress(i, total, "Apagando")
                full_path = os.path.join(self.current_path, item)
                try:
                    (shutil.rmtree if os.path.isdir(full_path) else os.remove)(full_path)
                    success_count += 1
                except:
                    error_count += 1

            self.progress_label.value = ""
            self.selected_items.clear()
            self.multiselect_mode = False
            self.set_status(f"✅ {success_count} itens apagados!" +
                          (f" ❌ {error_count} erros" if error_count else ""))
            self.refresh()

        self._confirm_delete(items_list, do_delete)

    def mark_for_move(self, item):
        self.item_to_move = os.path.join(self.current_path, item)
        self.item_to_copy = None
        self.set_status(f"📦 Item selecionado para mover: {os.path.basename(item)}")
        self.refresh()

    def mark_for_copy(self, item):
        self.item_to_copy = os.path.join(self.current_path, item)
        self.item_to_move = None
        self.set_status(f"📄 Item selecionado para copiar: {os.path.basename(item)}")
        self.refresh()

    def mark_multiple_for_move(self):
        self.item_to_move = [os.path.join(self.current_path, item) for item in self.selected_items]
        self.item_to_copy = None
        self._cleanup_multiselect()
        self.set_status(f"📦 {len(self.item_to_move)} itens selecionados para mover")
        self.refresh()

    def mark_multiple_for_copy(self):
        self.item_to_copy = [os.path.join(self.current_path, item) for item in self.selected_items]
        self.item_to_move = None
        self._cleanup_multiselect()
        self.set_status(f"📄 {len(self.item_to_copy)} itens selecionados para copiar")
        self.refresh()

    def _update_progress(self, current, total, action=""):
        """Atualiza contador de progresso"""
        self.progress_label.value = f"📊 {action}: {current}/{total}" if total > 1 else ""

    def _cleanup_multiselect(self):
        """Limpa estado de multiseleção"""
        self.multiselect_mode = False
        self.selected_items.clear()

    def _paste_multiple(self, items, operation):
        """Processa múltiplos itens para colar"""
        success_count = error_count = 0
        total = len(items)

        for i, src_path in enumerate(items, 1):
            self._update_progress(i, total, operation.title())
            dst = os.path.join(self.current_path, os.path.basename(src_path))
            try:
                if src_path == dst and operation == 'copy':
                    # Cria cópia com nome único
                    base_name = os.path.basename(src_path)
                    name, ext = os.path.splitext(base_name)
                    counter = 1
                    while os.path.exists(dst):
                        new_name = f"{name}_copia{counter}{ext}" if ext else f"{name}_copia{counter}"
                        dst = os.path.join(self.current_path, new_name)
                        counter += 1

                if src_path != dst or operation == 'copy':
                    if operation == 'move':
                        shutil.move(src_path, dst)
                    else:  # copy
                        (shutil.copytree if os.path.isdir(src_path) else shutil.copy2)(src_path, dst)

                success_count += 1
            except Exception:
                error_count += 1

        self.progress_label.value = ""
        return success_count, error_count

    def paste_item(self, btn=None):
        if isinstance(self.item_to_move, list):
            success, errors = self._paste_multiple(self.item_to_move, 'move')
            self.item_to_move = None
            self.set_status(f"✅ {success} itens movidos!" + (f" ❌ {errors} erros" if errors else ""))
        elif isinstance(self.item_to_copy, list):
            success, errors = self._paste_multiple(self.item_to_copy, 'copy')
            self.item_to_copy = None
            self.set_status(f"✅ {success} itens copiados!" + (f" ❌ {errors} erros" if errors else ""))
        elif self.item_to_move:
            try:
                self.set_status("⏳ Movendo...")
                shutil.move(self.item_to_move, os.path.join(self.current_path, os.path.basename(self.item_to_move)))
                self.item_to_move = None
                self.set_status("✅ Movido com sucesso!")
            except Exception as e:
                self.set_status(f"❌ Erro ao mover: {e}")
        elif self.item_to_copy:
            try:
                self.set_status("⏳ Copiando...")
                src = self.item_to_copy
                dst = os.path.join(self.current_path, os.path.basename(src))
                (shutil.copytree if os.path.isdir(src) else shutil.copy2)(src, dst)
                self.item_to_copy = None
                self.set_status("✅ Copiado com sucesso!")
            except Exception as e:
                self.set_status(f"❌ Erro ao copiar: {e}")
        self.refresh()

    def set_status(self, msg, duration=3):
        self.status_label.value = msg
        if duration > 0:
            threading.Timer(duration, lambda: setattr(self.status_label, 'value', '')).start()

    def _create_button(self, desc, callback, style='', width='auto'):
        """Cria botão com configurações padrão"""
        btn = widgets.Button(description=desc, button_style=style, layout=widgets.Layout(width=width))
        btn.on_click(callback)
        return btn

    def refresh(self):
        with self.output:
            clear_output()

            self._update_path_label()
            display(widgets.VBox([self.path_label, self.status_label, self.progress_label]))

            rows = []
            self.item_buttons = {}  # Limpa o cache de botões

            # Botão voltar
            if self.current_path != '/' and not self.multiselect_mode:
                rows.append(widgets.HBox([self._create_button('🔼 ..', lambda btn: self.go_up())]))

            # Botões de controle
            control_buttons = []
            if not self.multiselect_mode:
                control_buttons.append(self._create_button('📁➕ Nova Pasta', lambda btn: self.create_folder(), 'info'))
            else:
                control_buttons.extend([
                    self._create_button('☑️ Selecionar Tudo', lambda btn: self.select_all_items(), 'success'),
                    self._create_button('❌ Sair da Multiseleção', lambda btn: self.toggle_multiselect(), 'warning')
                ])

            if control_buttons:
                rows.append(widgets.HBox(control_buttons))

            # Botões de colar
            paste_configs = [
                (self.item_to_move, 'mover', 'success'),
                (self.item_to_copy, 'copiar', 'warning')
            ]

            for item, action, style in paste_configs:
                if item:
                    if isinstance(item, list):
                        desc = f'📥 Colar aqui ({action} {len(item)} itens)'
                    else:
                        desc = f'📥 Colar aqui ({action}): {os.path.basename(item)}'
                    rows.append(widgets.HBox([self._create_button(desc, self.paste_item, style)]))

            # Lista de itens
            items = self.list_directory(self.current_path)
            for item in items:
                if item.startswith("Error:"):
                    continue

                full_path = os.path.join(self.current_path, item)
                is_dir = os.path.isdir(full_path)

                if self.multiselect_mode:
                    is_selected = item in self.selected_items
                    label = f"{'☑️' if is_selected else '☐'} {'📁' if is_dir else '📄'} {item}"

                    # Usa closure para capturar o valor correto
                    def make_select_callback(item_name):
                        return lambda btn: self.toggle_item_selection(item_name)

                    select_btn = self._create_button(label, make_select_callback(item),
                                                   'info' if is_selected else '', '80%')
                    # Armazena referência do botão para atualização rápida
                    self.item_buttons[item] = select_btn

                    # Usa closure para o menu também
                    def make_menu_callback(item_name):
                        return lambda btn: self.show_item_menu(item_name)

                    menu_btn = self._create_button('...', make_menu_callback(item), '', '15%')
                else:
                    label = f"{'📁' if is_dir else '📄'} {item}"

                    # Usa closure para capturar valores corretos
                    def make_open_callback(item_name, is_directory):
                        return lambda btn: self.change_directory(item_name) if is_directory else None

                    def make_menu_callback(item_name):
                        return lambda btn: self.show_item_menu(item_name)

                    open_btn = self._create_button(label, make_open_callback(item, is_dir), '', '80%')
                    menu_btn = self._create_button('...', make_menu_callback(item), '', '15%')
                    select_btn = open_btn

                rows.append(widgets.HBox([select_btn, menu_btn]))

            display(widgets.VBox(rows))

    def show(self):
        display(self.output)

# Inicializar o File Manager
fm = FileManager()
fm.show()